In [1]:
using LinearAlgebra
using ITensors
using Plots

### Canonical Forms



In [2]:
# parameters 
N = 10
d = 3
D = 20

# random MPS
#=
    Order of legs: left-bottom-right
    Note: this is the conventional order used for MPSs in the code.
=#

Mrand = []
push!(Mrand, rand(1, d, D))
for l in 2:N-1
    push!(Mrand, rand(D, d, D))
end
push!(Mrand, rand(D, d, 1));


In [60]:
#=
    Function that takes an MPS 'M' as input and returns a copy of it that is
    transformed into left canonical form and normalized.
=#
function left_canonical(M::Vector{Array{Float64,3}})
    Mcopy = deepcopy(M)
    N = length(Mcopy)

    for l in 1:N-1
        A = Mcopy[l]
        (d1, d2, d3) = size(A)
        A_mat = reshape(A, d1 * d2, d3)
        U, S, Vt = svd(A_mat; full=false)
        chi = size(U, 2)
        Mcopy[l] = reshape(U, d1, d2, chi)

        # Propagate S*Vt into next tensor
        next_A = Mcopy[l+1]
        tmp = S * Vt
        tmpA = reshape(tmp, chi, 1, size(tmp, 2))  # add dummy middle leg for einsum
        # Contract tmp with next_A over last index of tmp and first of next_A
        Mcopy[l+1] = reshape(sum(tmpA .* permutedims(next_A, (2,3,1)), dims=3), size(tmpA,1), size(next_A,2), size(next_A,3))
    end

    return Mcopy
end


left_canonical (generic function with 3 methods)